In [71]:
from pandas import DataFrame, read_excel
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
#IMPORT MODELS
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.binary):
    """
    This function prints and plots the confusion matrix.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


# Loading data

In [72]:
training_list= ["./data/ase20.csv"]
training = pd.DataFrame()

for f in training_list:
    data = pd.read_csv(f, encoding='latin1',index_col=False)
    training = training.append(data, ignore_index=True)
    
    
training=training.iloc[:,:204]


In [73]:
#statistics about training samples
overfittingintrain = training[training['label']==1]
correctintrain=training[training['label']==0]
print("Total train: ", training.shape)
print("overfitting in train",overfittingintrain.shape)
print("correct in train",correctintrain.shape)
print("====================")


Total train:  (902, 204)
overfitting in train (654, 204)
correct in train (248, 204)


In [74]:
# 202 features
X_train = training.iloc[:,2:]
Y_train = training.iloc[:,1]


# process category features

In [75]:
X_train = pd.get_dummies(X_train)
X_train.head(2)
headers = X_train.columns
print(headers)
training.isnull().sum()
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan , strategy = 'most_frequent')
imputer = imputer.fit(X_train)
X_train = imputer.transform(X_train)

X_train = pd.DataFrame(X_train,columns=headers)

X_train.head(2)

Index(['patchedFileNo', 'addLineNo', 'rmLineNo', 'insertIfFalse', 'updIfFalse',
       'ifFalse', 'dupArgsInvocation', 'removeNullinCond', 'condLogicReduce',
       'insertBooleanLiteral',
       ...
       'S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ Class',
       'S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ Constructor',
       'S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ Do',
       'S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ For',
       'S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ ForEach',
       'S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ If',
       'S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ Method',
       'S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ Package',
       'S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ Try',
       'S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ While'],
      dtype='object', length=302)


,patchedFileNo,addLineNo,rmLineNo,insertIfFalse,updIfFalse,ifFalse,dupArgsInvocation,removeNullinCond,condLogicReduce,insertBooleanLiteral,...,S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ Class,S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ Constructor,S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ Do,S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ For,S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ ForEach,S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ If,S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ Method,S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ Package,S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ Try,S13_TYPE_OF_FAULTY_STATEMENT_BEFORE_1_ While
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
random_state = 42
sss = StratifiedKFold(n_splits=10, random_state=42, shuffle=False)
from sklearn import svm



# X_train = X_train.values
# Y_train = Y_train.values


print('Length of X (train): {} | Length of y (train): {}'.format(len(X_train), len(Y_train)))

# List to append the score and then find the average
accuracy_lst = []
precision_lst = []
recall_lst = []
f1_lst = []
auc_lst = []
tnsum = []
fpsum= []
fnsum= []
tpsum = []

rf = RandomForestClassifier(bootstrap=False, random_state=42)


log_reg_params = {}

rand_log_reg = RandomizedSearchCV(rf, log_reg_params, n_iter=10)

# Implementing SMOTE Technique 
# Cross Validating the right way
# Parameters
for train, test in sss.split(X_train, Y_train):
    pipeline = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), rand_log_reg) # SMOTE happens during Cross Validation not before..
    
    model = pipeline.fit(X_train[train], Y_train[train])
    best_est = rand_log_reg.best_estimator_
    prediction = best_est.predict(X_train[test])
    
    accuracy_lst.append(pipeline.score(X_train[test], Y_train[test]))
    precision_lst.append(precision_score(Y_train[test], prediction))
    recall_lst.append(recall_score(Y_train[test], prediction))
    f1_lst.append(f1_score(Y_train[test], prediction))
    auc_lst.append(roc_auc_score(Y_train[test], prediction))


    cnf_matrix = confusion_matrix(Y_train[test],prediction)
    tn, fp, fn, tp = cnf_matrix.ravel()
    tnsum.append(tn)
    fpsum.append(fp)
    fnsum.append(fn)
    tpsum.append(tp)


    
print('---' * 45)
print('')
print("accuracy: {}".format(np.mean(accuracy_lst)))
print("precision: {}".format(np.mean(precision_lst)))
print("recall: {}".format(np.mean(recall_lst)))
print("f1: {}".format(np.mean(f1_lst)))
print("tp: {}".format(np.sum(tpsum)))
print("tn: {}".format(np.sum(tnsum)))
print("fp: {}".format(np.sum(fpsum)))
print("fn: {}".format(np.sum(fnsum)))


print('---' * 45)

Length of X (train): 902 | Length of y (train): 902


/home/wasp/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/wasp/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/wasp/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/home/wasp/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 i

---------------------------------------------------------------------------------------------------------------------------------------

accuracy: 0.8913797313797314
precision: 0.9058874751702518
recall: 0.9495571095571096
f1: 0.9269271057952748
tp: 621
tn: 183
fp: 65
fn: 33
---------------------------------------------------------------------------------------------------------------------------------------
